In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from zipfile import ZipFile

In [3]:
# Descomprimir los archivos csv descargados de la página del DANE
# Solo se debe hacer una vez
censo_dir = r"microdatos_censo2018"
for d in os.listdir(censo_dir) :
    csv_dir = os.path.join(censo_dir,d,d.replace("_",""))
    csv_file = os.path.join(csv_dir,[z for z in os.listdir(csv_dir) if "_CSV" in z][0])

    with ZipFile(csv_file, 'r') as zObject:
        zObject.extractall(os.path.join(censo_dir,"unzip",d.split("_")[0]))

In [5]:
# Esta ruta debe ser la que tiene los archivos de los microdatos
censo_dir = r"microdatos_censo2018"
# Lista de los archivos de personas del censo
per_csv_list = [os.path.join(censo_dir,d,[c for c in os.listdir(os.path.join(censo_dir,d)) if "PER_" in c][0]) for d in os.listdir(censo_dir)]

In [ ]:
col_agg_per = ["U_DPTO","U_MPIO","UA_CLASE","P_SEXO","PA1_GRP_ETNIC","P_EDADR","CONDICION_FISICA"]

per_df = pd.DataFrame()
for c in tqdm(per_csv_list) :
    temp_df = pd.read_csv(c)
    temp_df = temp_df.groupby(col_agg_per,dropna=False)["COD_ENCUESTAS"].agg("count")
    temp_df.name = "N"
    per_df = pd.concat([per_df,temp_df.reset_index()])
    per_df.to_csv("aux_files/censo_2018/personas_agregado_censo_2018.csv",index=False)